# Predicting GDP as a function of a country's ruggedness
This practical helps you answer a fictitious geographer's questions to you, the data scientist. This practical is based on a paper by [Nunn and Puga, Review of Economics and Statistics, 2012](https://diegopuga.org/research/rugged.pdf). Once you're done with the practical, check out the paper to see how the authors propose to explain the results of their statistical analysis. The solution of the practical is also avalaible at https://colab.research.google.com/drive/1jmhhnHWZ1ayGumizGV06bgZpHMdVd0mJ?usp=sharing .

In [ ]:
# make sure the notebook reloads the module each time we modify it
%load_ext autoreload
%autoreload 2

# make sure the displays are nice
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

In [ ]:
import solved_gdp_prediction as gp
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="ticks")
import numpy as np
import numpy.linalg as npl
import scipy.stats as sps
import pymc3 as pm
import arviz as az

## Preparing data and utilities

In [ ]:
# I prepared a class that fetches data for you
GP = gp.GDPPredictionUsingLinearRegression()
X, y = GP.fetch_data()
print("The dimensions of data are", X.shape)

# Data is normalized
print("Mean and std of y are", np.mean(y), np.std(y))

# Let us plot data
sns.scatterplot(X, y)
plt.ylabel("log GDP")
plt.xlabel("Ruggedness index")
plt.show()

Say you work for a geographer. She wants to test the hypothesis that the log GDP increases with ruggedness. You reply that without knowledge about how $X$ and $y$ relate, and only 170 points, it seems reasonable to stick to a simple model, say linear regression. 

**Question:** Assuming you perform a linear regression of the log GDP onto the ruggedness index, how will you formalize the test that the geographer is asking for? In other words, what are the states, actions, loss function, and Bayes action? What quantity should you report to the geographer?

**Solution:** As Bayesians, we need a distribution over states, a set of actions, and and a loss function.
1. Linear regression and a choice of prior will give us a joint distribution $\pi$ over states $s=(X, \mathbf{y}, \theta)$, where $\theta=(\alpha,\beta,\sigma)$, $y_i=\alpha+\beta x_i+\epsilon_i$, and, for example, $\epsilon_i\sim\mathcal{N}(0,\sigma^2)$ i.i.d. 
2. Actions are naturally indexed by predictors for the sign of $\beta$. Let $\mathcal G$ be the set of measurable functions that map values of $X,\mathbf y$ to $\{0,1\}$, where we think of a prediction of $1$ as predicting that $\beta>0$. Then, if we take
$$
\mathcal A = \{a_g: s\mapsto (1_{\beta>0}, g(X,y)), g\in \mathcal G \},
$$
we shall be able to model type I and type II errors. 
The geographer should tell us how much she would suffer if we answered yes/no and the state was $s$. This gives us costs $\ell_1, \ell_2>0$. We can then take
$$
L(a,s) = \ell_1 1_{\beta>0} \times (1-g(X,\mathbf y)) + \ell_2 1_{\beta\leq 0} \times g(X,\mathbf y).
$$
Like in asymmetric classification (see exercise sheet), the Bayes action is then to set $g^*(X,\mathbf y) = 1 $ if and only if
$$ 
\pi(\beta>0\vert X,\mathbf{y}) > \frac{\ell_2}{\ell_1+\ell_2}.
$$
The quantity to report to the geographer is thus $\pi(\beta>0\vert X,\mathbf{y})$.

## 1. Linear regression

**Question:** First write down your model (likelihood and prior).

**Solution:**
* The likelihood is $p(\mathbf{y}\vert X, \theta) = \prod_{i=1}^n \mathcal{N}(y_i\vert \alpha+\beta x_i, \sigma^2)$.
* The simplest prior is $p(\theta) = p(\alpha,\beta,\log\sigma) = \mathcal{N}((0,0,0),I)$, in which case we actually know the posterior in closed form; see Lecture #1. We shall also distributions with fatter tails, like a Student or a Cauchy, in which case we genuinely need numerical integration.
* The posterior is proportional to 
$$ p(\theta\vert X,y) \propto p(\mathbf{y}\vert X, \theta)p(\theta).$$

**Question:** Now use PyMC to code your model in the companion Python file, within method `get_mcmc_sample` of class `GDPPredictionUsingLinearRegression.`and method . Then run the default MCMC sampler (HMC with NUTS, we shall cover it in the next lecture). If your syntax is correct, you should be able to run the following cells to visualize the results. Comment on the realization of your chain. What is the `Rhat` column in the summary of the trace?

In [ ]:
trace = GP.get_mcmc_sample()

In [ ]:
pm.summary(trace)

In [ ]:
az.plot_trace(trace, show=True)
plt.show()

**Question:** Now turn your chain into an estimator of $\pi(\beta>0\vert X, \mathbf{y})$. 

In [ ]:
beta_traces = trace['beta']
plt.hist(beta_traces)
plt.show()
print("Our estimate of the posterior probability of beta>0 is", np.mean(beta_traces>0))

**Solution:** 
* If $\ell_1=\ell_2$, it's a close call: if you resample your chain and or change your prior, you'll see that our estimator has a standard deviation of the order of the difference between its mean and $1/2$. Depending on the realization of your MCMC chain and you choice of prior, you will thus recommend a different action, and it might not be the underlying Bayes action. This means that we need more MCMC iterations to reduce MC error, or more data, or a different model/prior.
* If $\ell_1 = 2\ell_2$, for instance, then we are confident that $\pi(\beta>0\vert X, \mathbf{y})>1/3$ and the Bayes action is to conclude that $\beta>0$.
* If $\ell_2 = 2\ell_1$, for instance, then we are confident that $\pi(\beta>0\vert X, \mathbf{y})<2/3$ and the Bayes action is to conclude that $\beta<0$. In particular, see how our final decision depends on the loss function, not only on the posterior.

## 2. A hierarchical linear regression

The same geographer comes back to you. She has a new hypothesis: GDP only increases with ruggedness for African nations. We have the same 170 points, with $y_i$ the log GDP, and $x_i$ the ruggedness index, for $i=1,\dots,170$. But now we also have a new binary variable $z_i\in\{0,1\}$, $i=1,\dots,170$, that indicates whether the corresponding country is in Africa. 

In [ ]:
# I prepared a new class that fetches data for you
HM = gp.GDPPredictionUsingHierarchicalRegression()
X, y = HM.fetch_data()

# Let us plot data
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6), sharey=True)

sns.scatterplot(X["rest"], y["rest"], ax=ax[0])
ax[0].set(xlabel="Ruggedness index", ylabel="log GDP", title="Non-African nations")

sns.scatterplot(X["African"], y["African"], ax=ax[1])
ax[1].set(xlabel="Ruggedness index", ylabel="log GDP", title="African nations")

plt.show()

This time we want to examine a potential difference between the parameters corresponding to African and non-African countries. We thus come up with a hierarchical model 
* $y_{\circ,i}=\alpha_\circ+\beta_\circ x_i+\epsilon_{\circ,i}$, where $\epsilon_{\circ,i}$ are i.i.d. $ \mathcal{N}(0,\sigma_\circ^2)$ and $\circ\in\{\text{African}, \text{rest}\}$,
* $\alpha_{\text{African}}, \alpha_{\text{rest}} \sim p(\cdot \vert \alpha)$ i.i.d., with some prior on the common $\alpha$. Same for $\beta_\circ, \sigma_\circ$ and $\beta, \sigma$. In total, we know have 9 parameters: three per class, and three common ones.

Go to class `GDPPredictionUsingHierarchicalRegression` and fill in the method `get_mcmc_sample` with your model. Make simple choices for priors, and make sure to later check that your decision does not depend on your choices. Run your MCMC sampler and examine the output of your chain.

In [ ]:
trace_hierarchical = HM.get_mcmc_sample(5000)

In [ ]:
pm.summary(trace_hierarchical)

See how the HPD credible interval for $\beta_\text{African}$ has shifted to the right compared to the non-hierarchical model.

In [ ]:
az.plot_trace(trace_hierarchical)
plt.show()

In [ ]:
# There's a strong difference between the posterior marginals of beta_African and beta_rest. 
# Meanwhile the posterior on their common mean beta is quite uninformative.
plt.hist(trace_hierarchical['beta_African'], label="African")
plt.hist(trace_hierarchical['beta_rest'], label="rest")
plt.hist(trace_hierarchical['beta'], alpha=.3)
plt.legend()
plt.show()

**Question:** what do we conlude and tell the geographer?

In [ ]:
# Again, we estimate the posterior of beta_African being positive
print(np.mean(trace_hierarchical['beta_African']>0))

**Solution:** This time, unless the ratio $\ell_1/\ell_2$ is extreme, we will conclude that indeed, the GDP of African countries increases with ruggedness. Run your chain again with different priors and check that this decision is robust to prior changes.

**Bonus question:** The geographer asks you for a prediction of the GDP of an African country with ruggedness $4$, what do you say?

**Answer:** It is again a decision problem. As seen in course, if your loss for the prediction is $L^2$, then the Bayes answer is the mean of the posterior predictive. If the geographer prefer an interval, you can report a credible interval, i.e. an interval with posterior predictive mass larger than a given threshold, say $95\%$. In any case, you need to use your chain to estimate the corresponding prediction or interval. Below I plot the estimated posterior predictive in the form of a sample. To find the credible interval corresponding to a ruggedness of 4, simply draw a vertical line at abscissa 4, and report an interbal containing 95% of the samples.

In [ ]:
sns.scatterplot(X["African"], y["African"])
plt.ylabel("log GDP")
plt.xlabel("Ruggedness index")
t = trace_hierarchical
lag = 100
for a, b in zip(t["alpha_African"][::lag], t["beta_African"][::lag]):
    plt.plot(X["African"], a + b*X["African"], color="orange", alpha=.1)
plt.show()